Dev notes:
- through 2022/02/28, developed as "MetadataUpdater_v01" in D:\ArcGIS\Projects\CM1 and Metadata Update
- from 2022/03/01, developed as (GitHub)/metadata/MetadataUpdater

2022/03/01: v00.5

Potential future work:
- Load enumerated domain

Setup & Prerequisites:
0. Requires ArcGIS Pro. Developed & tested at ArcGIS Pro 2.9.1 in the defualt python env. Run as Notebook (.ipynb) in ArcGIS Pro.
1. Acquire & stage data. Should be a copy of the most recent source data, not the source data itself. This process may involve changing geometry, attributes, and table schema.
2. Run "Check Geometry" tool. Use caution when deleting null geometry. Evaluate outputs and note necessary repairs.
3. Run "Data Inventory" script tool.
4. Complete data inventory and make an necessary geometry, attribute, or table updates. This may include repairing geometry errors found in step 2 and/or loading a new copy of the data in a fresh table schema.
5. Run "Check Geometry" tool again (still using caution when deleting null geometry) and run "Repair Geometry" if necessary.
6. Run "Recalculate Feature Class Extent" tool.
7. **Run "Metadata Updater" script tool. Output will be a near-complete XML metadata file.**
8. Use the "SpatRef_supplement" script tool and Section 1 and Section 2 worksheets to complete the metadata document.
9. Review and create HTML copy. Publish both copies.

## 1. Inputs:

- target: Paste FULL PATH to feature class for which metadata is being created.
- workingfolder: Paste FULL PATH to folder where output will be saved.
- inventory: Paste FULL PATH to Excel table with Data Inventory.
- mrcog_template: Paste FULL PATH to MRCOG metadata template XML.

In [ ]:
# Paste full file path, not just layer name

# Target feature class for metadata
target = r''

# Folder to save XML metadata file
workingfolder = r''

# Data inventory with information to load into metadata
inventory = r'.xlsx'

# Path to MRCOG metadata template file
mrcog_template = r'.xml'

from datetime import datetime
start_time = datetime.now()
print('''
{}
Ready to create metadata...
For: {}
From: {}
In: {}
'''.format(start_time, target, workingfolder, inventory)
)

## 2. Automated Metadata Update

Run remaining cell. See messages at bottom.

In [ ]:
import xml.etree.ElementTree as ET
from os import path
import openpyxl
from string import ascii_uppercase

# Docs: metadata template https://pro.arcgis.com/en/pro-app/latest/arcpy/metadata/metadata-class.htm#M2_GUID-ED041163-664D-4982-9CBC-86A9F2332D11
template_md = arcpy.metadata.Metadata(mrcog_template)
md = arcpy.metadata.Metadata(target)
md.copy(template_md)
md.save()

md.synchronize('ALWAYS')

fcname = path.split(target)[1]
workxml = path.join(workingfolder, f'{fcname}.xml')
md.exportMetadata(workxml, 'FGDC_CSDGM', 'REMOVE_ALL_SENSITIVE_INFO')

wb = openpyxl.load_workbook(filename=inventory)
ws = wb['Sheet']

columnDict = dict(zip(range(26), ascii_uppercase))

x = ''
headRow = 0
while x != 'Field Name':
    headRow += 1
    x = ws[f'A{headRow}'].value

schema = []
for col in range(ws.max_column):
    c = columnDict[col]
    schema.append(ws[f'{c}{headRow}'].value)

f = columnDict[schema.index('Field Name')]
d = columnDict[schema.index('Definition')]
s = columnDict[schema.index('Def. Source')]
dom = columnDict[schema.index('Domain Type')]
mn = columnDict[schema.index('Minimum')]
mx = columnDict[schema.index('Maximum')]
u = columnDict[schema.index('Range Units')]
n = columnDict[schema.index('Domain Notes')]

a = headRow + 1
b = ws.max_row + 1

defs = {
    ws[f'{f}{i}'].value: ws[f'{d}{i}'].value
    for i in range(a, b)
}

sources = {
    ws[f'{f}{i}'].value: ws[f'{s}{i}'].value
    for i in range(a, b)
}

doms = {
    ws[f'{f}{i}'].value: ws[f'{dom}{i}'].value
    for i in range(a, b)
}

minmax = {
    ws[f'{f}{i}'].value: [ws[f'{mn}{i}'].value, ws[f'{mx}{i}'].value]
    for i in range(a, b)
    if 'n/a' not in [ws[f'{mn}{i}'].value, ws[f'{mx}{i}'].value]
}

units = {
    ws[f'{f}{i}'].value: ws[f'{u}{i}'].value
    for i in range(a, b)
}

notes = {
    ws[f'{f}{i}'].value: ws[f'{n}{i}'].value
    for i in range(a, b)
}


tree = ET.parse(workxml)
root = tree.getroot()

firstlevel = [
    ['attrdef', defs],
    ['attrdefs', sources]
]

for attr in root.iter('attr'):
    label = attr[0].text
    if label in defs.keys():
        tags = [i.tag for i in attr]
        for t, s in firstlevel:
            if t not in tags and s[label] and s[label] != 'n/a':
                ET.SubElement(attr, t).text = s[label]
        if 'attrdomv' not in tags and doms[label] and doms[label] != 'n/a':
            ET.SubElement(attr, 'attrdomv')
        if len(attr) == 4:
            domain = attr[3]
            if doms[label] == 'Range':
                ET.SubElement(domain, 'rdom')
                ET.SubElement(domain[0], 'rdommin').text = str(
                    minmax[label][0]
                    )
                ET.SubElement(domain[0], 'rdommax').text = str(
                    minmax[label][1]
                    )
                ET.SubElement(domain[0], 'attrunit').text = units[label]
            elif doms[label] == 'Unrepresentable':
                ET.SubElement(domain, 'udom').text = notes[label]
            elif doms[label] == 'Enumerated':
                ET.SubElement(domain, 'edom')
            elif doms[label] == 'Codeset':
                ET.SubElement(domain, 'codesetd')

tree.write(workxml)

cell_time = datetime.now()
print('{}. Complete.'.format(cell_time))


### Map of attributes (attr) in metadata xml:

Index:  Tag:        Text:<br>
0       attrlabl    (field name)<br>
1       attrdef     (definition)<br>
2       attrdefs    (source)<br>
3       attrdomv    <br>

#### Map of attrdomv (attr\[3\])
0   rdom<br>
0,0     rdommin<br>
0,1     rdommax<br>
0,2     attrunit<br>
<br>
OR<br>
<br>
0   udom    Text:(description)<br>
<br>
OR<br>
<br>
0   edom<br>
0,0     edomv   Text:(value)<br>
0,1     edomvd  Text:(value definition)<br>
0,2     edomvds Text:(val def source)<br>
1   edom (structure repeats for each enumerated value)<br>

#### coded domain TBD